---
author: Delvin Mackenzie
date: 11/19/2019
contact: delvin@uw.edu

---



In [2]:
#Math stuff
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np
import math  

#Visualization stuff
import seaborn as sns
#sns.set(style="darkgrid")
sns.set(style="whitegrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

#ML stuff
from sklearn import preprocessing

In [1]:
#create merge.csv

In [6]:
#check list of files in data 
directory = "./data/dedicated_host/merge_data/"
print(os.listdir(directory))

['newdata_iperf_m5d_48 - newdata_iperf_m5d_48.csv', 'm5d-no-idle-dedicated-host-11-07-2019 - newdata.csv', 'y_cruncher_08_22_2019 - y_cruncher_08_22_2019.csv', 'm5d_48vms_pgbench_no_idle_11_11_2019 - Data.csv']


In [82]:
#import the 4 benchmarks
data_sysbench = 'm5d-no-idle-dedicated-host-11-07-2019 - newdata.csv'
data_iperf = 'newdata_iperf_m5d_48 - newdata_iperf_m5d_48.csv'
data_y_cruncher = 'y_cruncher_08_22_2019 - y_cruncher_08_22_2019.csv'
data_pgbench ='m5d_48vms_pgbench_no_idle_11_11_2019 - Data.csv'

In [92]:
#TODO: configure header values accordingly...
df_sysbench = pd.read_csv(directory + data_sysbench, header = 1)
df_iperf = pd.read_csv(directory + data_iperf, header = 0)
df_y_cruncher = pd.read_csv(directory + data_y_cruncher, header = 0)
df_pgbench = pd.read_csv(directory + data_pgbench, header = 1)

In [125]:
#y-cruncher typo in header...adjust it...
df_y_cruncher['setId'] = df_y_cruncher['setid']

In [141]:
def cleanAndReturnDf(df, experiment_type, col, n_rows = 3, vm_count = 48):
    ''' returns cleaned df with n rows sampled randomly. 
        also drops the first row of each run since they are often outliers.
    '''
    
    df = df[col]
    df = df.dropna()
    
    #sort by setId and instanceID
    df = df.sort_values(by = ['setId','instanceID'], ascending=False)
    
    #iperf doesn't need so skip this part
    if (experiment_type != 'iperf'):
        
        df['vmId'] = pd.factorize(df.instanceID)[0] + 1 #replace instanceID with numerical vmID
        
        df = df.sort_values(by = ['setId','vmId'])
        df = df.drop(columns=['instanceID'])
        
        #finally reverse setId to match eds version! 
        df['setId'] = pd.factorize(df.setId)[0]- vm_count + 1
        #absolute value
        df['setId'] = df['setId'].abs()

        #add + 1 for off by 1 error
        df['setId'] += 1

    df = df.sort_values(by = ['setId','vmId'])
    df = df.reset_index(drop=True)

    #set nth column #change depending on data set...
    if (experiment_type == 'iperf'):
        nth_column = 11
    elif (experiment_type != 'pgbench'):
        nth_column = 10
    
    #drop every nth row
    if (experiment_type != 'pgbench'):
        df = df[df.index % nth_column != 0]
        
        #sample of n_rows to match pgbench row count
        df = df.groupby(['setId', 'vmId']).apply(pd.DataFrame.sample, n=n_rows, random_state=19).reset_index(drop=True)
    
    return df

In [142]:
#created cleaned dataframes for each benchmark
df_iperf_clean = cleanAndReturnDf(df_iperf, 'iperf', ['Total','setId','vmId', 'instanceID'])
df_sysbench_clean = cleanAndReturnDf(df_sysbench, 'sysbench', ['setId', 'wallTime', 'instanceID'])
df_pgbench_clean = cleanAndReturnDf(df_pgbench, 'pgbench', ['setId', 'transactions', 'instanceID'])
df_y_cruncher_clean = cleanAndReturnDf(df_y_cruncher, 'y-cruncher', ['setId','wallTime','instanceID'])


In [143]:
#create df for merge.csv
df_merge = df_sysbench_clean

In [149]:
df_merge = df_merge.rename(columns={"wallTime": "sysbench"})

In [152]:
df_merge['iperf'] = df_iperf_clean['Total']
df_merge['pgbench'] = df_pgbench_clean['transactions']
df_merge['ycruncher'] = df_y_cruncher_clean['wallTime']

In [156]:
col = ['iperf', 'sysbench', 'ycruncher', 'pgbench', 'setId', 'vmId']
df_merge = df_merge[col]

In [157]:
df_merge.head()

,iperf,sysbench,ycruncher,pgbench,setId,vmId
0,18.32,7.172616,4.384688,185811,1,1
1,18.35,7.172229,4.371897,185256,1,1
2,18.32,7.172506,4.415437,185510,1,1
3,17.26,7.176181,4.444210,185058,2,1
4,17.6,7.172425,4.462397,184964,2,1


In [159]:
df_merge.describe()

,sysbench,ycruncher,setId,vmId
count,3528.000000,3528.000000,3528.000000,3528.000000
mean,8.461232,5.763750,32.333333,16.666667
std,0.566776,0.588269,11.427711,11.427711
min,7.172215,4.371897,1.000000,1.000000
25%,8.004919,5.454348,24.000000,7.000000
50%,8.773572,5.597796,34.000000,15.000000
75%,9.045796,6.372122,42.000000,25.000000
max,9.187299,6.933911,48.000000,48.000000


In [160]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3528 entries, 0 to 3527
Data columns (total 6 columns):
iperf        3528 non-null object
sysbench     3528 non-null float64
ycruncher    3528 non-null float64
pgbench      3528 non-null object
setId        3528 non-null int64
vmId         3528 non-null int64
dtypes: float64(2), int64(2), object(2)
memory usage: 165.5+ KB


In [161]:
#print to csv
df_merge.to_csv(path_or_buf= directory + 'merged.csv')

In [52]:
# col = ['setId', 'wallTime', 'instanceID']

In [58]:
#filter by relevant columns
# df = df_sysbench[col]

In [61]:
# df.dropna(inplace=True)

In [59]:
#sort by setId and instanceID
# df.sort_values(by = ['setId','instanceID'], ascending=False, inplace=True)

In [60]:
#replace instanceID with numerical vmID
# df['vmId'] = pd.factorize(df.instanceID)[0] + 1

In [62]:
#sort by vmId then setId 
# df.sort_values(by = ['setId','vmId'], inplace=True)

In [64]:
#drop instanceID
# df.drop(columns=['instanceID'], inplace=True)

In [68]:
# #finally reverse setId to match eds version! 
# #change 48 if necessary to match vm count
# vm_count = 48
# df['setId'] = pd.factorize(df.setId)[0]- vm_count + 1
# #absolute value
# df['setId'] = df['setId'].abs()

In [72]:
#add + 1 for off by 1 error
# df['setId'] += 1

In [73]:
# df.sort_values(by = ['setId','vmId'], inplace=True)
# df.reset_index(drop=True, inplace=True)

In [75]:
# experiment_type = 'sysbench'

In [76]:
# #drop every nth row...
# #set nth column #change depending on data set...
# if (experiment_type == 'iperf'):
#     nth_column = 11
# #     if (not dedicated_host_mode):
# #         df_data = df_data[:551] #drop every run except first run
# elif (experiment_type != 'pgbench'):
#     nth_column = 10

In [77]:
# #drop every nth column
# #drop first row of each vm run since they are outliers (except pgbench)
# if (experiment_type != 'pgbench'):
#     df = df[df.index % nth_column != 0]

In [79]:
# #take a sample of 3 for the 3 bench marks...
# #drop all but 3 rows? since pgbench only has 3 rows?
# n_rows = 3
# df = df.groupby(['setId', 'vmId']).apply(pd.DataFrame.sample, n=n_rows, random_state=19).reset_index(drop=True)

In [158]:
# df.head()

In [ ]:
#drop all but 3 rows? since pgbench only has 3 rows?
#df.groupby('Group_Id').apply(pd.DataFrame.sample, n=1).reset_index(drop=True)

In [ ]:
#export to csv